<a href="https://colab.research.google.com/github/ccasadei/corso-ia/blob/master/04RetiNeurali/03TransferLearning/transfer_learning_keras01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning in Keras
## Esercizio 1
**Author: Cristiano Casadei**

In [1]:
!git clone https://github.com/ccasadei/dataset_transferlearning01.git ./dataset

Cloning into './dataset'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 127 (delta 1), reused 126 (delta 0), pack-reused 0
Receiving objects: 100% (127/127), 18.22 MiB | 5.27 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
from tensorflow.keras.applications import MobileNetV2, mobilenet_v2
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from math import ceil
import numpy as np

In [3]:
# Definisco un modello 'backbone' utilizzando il classificatore 'MobileNet'
# pre-addestrato con dataset 'ImageNet'
# Indico che non voglio i top layer del classificatore, perchè userò i miei
# Indico anche che l'ultimo layer sarà un GlobalAveragePool, che consente di
# limitare ulteriormente i parametri da gestire
backbone = MobileNetV2(weights='imagenet', include_top=False, pooling="avg")

# indico i layer del backbone come "non addestrabili", in modo da non modificarli
for l in backbone.layers:
    l.trainable = False

# definisco gli ultimi layer di classificazione, usando come ingresso le uscite del backbone
x = backbone.output
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

# il mio modello complessivo avrà gli stessi ingressi del backbone e l'uscita che ho definito
# nei mei top-layers
model = Model(inputs=backbone.input, outputs=preds)

# visualizzo un sommario del modello complessivo
model.summary()

2022-12-19 13:18:40.312389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 13:18:40.317653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 13:18:40.317808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 13:18:40.318493: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, None, 3 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [4]:
# compilo il modello con una loss di classificazione, l'ottimizzatore Adam ed aggiungendo l'accuracy come metrica
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["accuracy"])

# creo un generatore di immagini che utilizzi la funzione di preprocessing necessaria al modello MobileNetV2
train_datagen = ImageDataGenerator(preprocessing_function=mobilenet_v2.preprocess_input)

# indico al generatore di immagini dove si trovano le immagini, le dimensioni da usare, il formato colore da usare,
# il batch_size con cui costruire i vari batch, il tipo di classificazione, e se deve mischiare il dataset
train_generator = train_datagen.flow_from_directory('./dataset/train',
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 116 images belonging to 2 classes.


/opt/conda/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [5]:
# addestro il modello usando il generatore di immagini definito in precedenza, indicando
# quanti cicli eseguire per ogni epoca (lo calcolo dividendo l'ampiezza del dataset per il batch_size)
# ed utilizzando 10 epoche in tutto
model.fit_generator(generator=train_generator,
                    steps_per_epoch=ceil(train_generator.n / train_generator.batch_size),
                    epochs=10,
                    verbose=1)

/opt/conda/lib/python3.9/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-12-19 13:18:42.351436: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-12-19 13:18:44.867963: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-12-19 13:18:45.421920: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-19 13:18:45.422319: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-19 13:18:45.422350: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2022-12-19 13:18:45.422752: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-19 13:18:45.422821: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


4/4 [==============================] - 5s 310ms/step - loss: 0.5668 - accuracy: 0.7586
Epoch 2/10
4/4 [==============================] - 2s 346ms/step - loss: 0.1971 - accuracy: 0.9828
Epoch 3/10
4/4 [==============================] - 1s 338ms/step - loss: 0.0701 - accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 1s 398ms/step - loss: 0.0269 - accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 1s 353ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 1s 363ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 1s 386ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 8/10
4/4 [==============================] - 1s 378ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 9/10
4/4 [==============================] - 1s 422ms/step - loss: 9.3269e-04 - accuracy: 1.0000
Epoch 10/10
4/4 [==============================] - 1s 389ms/step - loss: 7.0226e-04 - accuracy: 1.0000


In [6]:
def load_image(img_path):
    # carico l'immagine dal file
    img = image.load_img(img_path, target_size=(224, 224))
    # trasformo l'immagine in un array Numpy
    # lo shape dell'array sono (altezza, larghezza, canali colore)
    # quindi in questo caso (224, 224, 3)
    img_array = image.img_to_array(img)
    # aggiungo una dimensione all'inizio
    # lo shape diventa (1, 224, 224, 3)
    # dove "1" indica quante immagini sono presenti nel batch
    img_array_batch = np.expand_dims(img_array, axis=0)
    # normalizzo i valori da 0..255 a 0..1
    img_array_batch /= 255.

    return img_array_batch

In [7]:
# verifico la predizione su un disegno di ciciarella (immagine non utilizzato in training)
img_di_test = load_image("./dataset/test/cinciarella.jpg")
predizione = model.predict(img_di_test)
print("Predizione cinciarella:", predizione)

Predizione cinciarella: [[0.8885327  0.11146729]]


In [8]:
img_di_test = load_image("./dataset/test/corvo.jpg")
predizione = model.predict(img_di_test)
print("Predizione corvo:", predizione)

Predizione corvo: [[2.5286013e-04 9.9974710e-01]]
